In [2]:
import sys

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as dd

import string
import itertools

import processHist
import utils
import processZephyr

ctry = 'GB'

fd_base = '' # replace with your dir
bvd_fd = fd_base + 'bvd/'
sub_fd = fd_base + 'bvd_sub/'
header_fd = fd_base + 'bvd_header/'
bvd_proc_fd = fd_base + 'bvd_qualified/'
ind_fd = fd_base + 'bvd_full_header/'
output_fd = 'output/'
hist_fd = fd_base + 'bvd_hist_2020_06/'
fin_hist_fd = hist_fd +  'Financials/'
des_hist_fd = hist_fd + 'Descriptive/'
own_hist_fd = hist_fd + 'Ownership/'
zephyr_fd = fd_base + 'Zephyr/'

# Preparation
1. In the server, create a main folder, then create 3 folders inside, e.g.: 
    - bvd_hist_2020_06/
        - Ownership/
        - Financials/
        - Descriptive/
2. For Ownership, Copy Links_2007.rar - Links_2019.rar, Links_current.rar, and Entities.rar to /Ownership.
3. Unzip all the files in Ownership. Rename Links_current.txt to Links_2020.txt. Put all rar files to Ownership/archived. Create a folder, e.g. Ownership/GB, for the target country.
4. For Descriptive, copy all Legal_info.part*.rar and Industry_classification.part*.rar to the server /Descriptive. Unzip and put all rar files to Descriptive/archived.
5. For Financials, copy all Industry-Global_financials_and_ratios-USD.part*.rar to /Financials. Unzip and put all rar files to Financials/archived.

# Read historical financial data and extract the country
This will create Financials/GB_fin_hist.csv, which contains all the data for that country.

In [ ]:
processHist.readHistFin(ctry)

# Read historical header info
This will create Descriptive/GB/GB_sup_header, which contains the essential header information for firms that are not included in WRDS.

In [ ]:
processHist.readHistHeader(ctry)

# Merge historical financial with WRDS financial
#### getMergedFinSet(ctry):
1. Apply the filter df['empl']>=10) | (df['toas']>=1000000) | (df['opre']>=250000) to the historical data and find the list of firms.
2. Apply the same filter to WRDS (2011-2020) and get the list of firms.
3. Get the union set of both data and save it as Financials/GBfilter_1_merge_set.csv.
4. For the historical data, extract the firms in the union set and save it as Financials/GB_fin_hist_filter1_rename.csv.
5. Number of companies including (hist, hist after LF, hist after filter 1, wrds, wrds after LF, wrds after filter 1) at different stages are stored in output/GB_merge_wrds_hist_filter1.csv

#### combineFin(ctry):
5. For the WRDS data, extract the firms in the union set.
6. Combine the extracted historical data and the WRDS data.
7. Use ['bvdid', 'conscode', 'filing_type', 'closdate'] as a key to remove redundancy. Use the WRDS data if it is available since it is more updated.
8. Save it as Financials/GB_filter1_merge.csv.

In [ ]:
processHist.getMergedFinSet(ctry)

In [ ]:
processHist.combineFin(ctry)

/tmp/ipykernel_1779336/1526553785.py:1: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.combineFin(ctry)


2008


/tmp/ipykernel_1779336/1526553785.py:1: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.combineFin(ctry)


2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


/tmp/ipykernel_1779336/1526553785.py:1: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.combineFin(ctry)


#### consolidateFin(ctry)
1. remove limited financials
2. round date to the closest month start date so that nearby dates can cluster
3. remove conscode duplicates for bvdid-closdate pairs, e.g. multiple U1
4. for U1-U2 and C1-C2 duplicates, select the one with fewer nans
5. remap U1/U2/C1/C2. If a company has only U or C, it will be U1/C1. Otherwise, it will be C2 and U2.
6. remove U2 since we already have C2
7. save the df to Financials/GB/GB_cons.csv

In [ ]:
processHist.consolidateFin(ctry)

/tmp/ipykernel_1782877/1413070190.py:1: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.consolidateFin(ctry)


Before merging unconsolidate and consolidate
U1    12630018
U2      533296
C2      325907
C1      201180
Name: conscode, dtype: int64
Removing conscode duplicates using number of nans
U1    12566886
U2      530880
C2      324752
C1      183565
Name: conscode, dtype: int64
Number of U1-U2 9093
Number of C1-C2 19209
Number of U1-U2 duplicates 9093
Number of C1-C2 duplicates 19209
U1    12561057
U2      527616
C2      308890
C1      180218
Name: conscode, dtype: int64
Before remapping, number of single U2: 519638
After remapping, number of single U2: 0
U1    13076201
C1      476636
U2       12472
C2       12472
Name: conscode, dtype: int64


In [ ]:
processHist.chooseReportInYear(ctry)

/tmp/ipykernel_1782877/2020650078.py:1: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.chooseReportInYear(ctry)


In [ ]:
processHist.expandReportYear(ctry, start_yr=1990)

/tmp/ipykernel_1804745/3915028386.py:1: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.expandReportYear(ctry, start_yr=1990)
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:304: DtypeWarning: Columns (6,8,14,22,25,27,36) have mixed types.Specify dtype option on import or set low_memory=False.
  df_header = readWRDSHeader(ctry)


Before expanding: 13474733


# Read historical industry information and merge it with WRDS (TODO)
1. read industry info from WRDS (currently in bvd_full_header/GB_*.csv, *=l/m/s)
2. read industry info from the historical data (Descriptive/Industry_classifications.txt) and extract the country
3. get the remaining set from the historical data
4. remove those that do not have industry info
5. save as Descriptive/GB/GB_sup_industry.csv

In [ ]:
### outdated - processHist.readIndustries(ctry)

In [ ]:
# processHist.readIndustries(ctry, 'naics')

In [ ]:
# processHist.readIndustries(ctry, 'sic')

In [2]:
processHist.readIndustries(ctry, 'uksic')

/tmp/ipykernel_2718362/508598755.py:1: DtypeWarning: Columns (5,12,18,24) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.readIndustries(ctry, 'uksic')


In [8]:
processHist.mergeIndustries(ctry)

/tmp/ipykernel_2718362/2435716040.py:1: DtypeWarning: Columns (8,105,107,109,113,115,117,119) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.mergeIndustries(ctry)
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [3]:
processHist.getEntityIndustries(ctry)

In [10]:
processHist.findEntitiesGPLP()

In [11]:
processHist.getNanSICfromPanel()

/tmp/ipykernel_2718362/191264972.py:1: DtypeWarning: Columns (6,8,105,107,109,113,115,117,119,129,135,136) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.getNanSICfromPanel()


In [12]:
processHist.pvNameMatch()

/tmp/ipykernel_2718362/1791768742.py:1: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.pvNameMatch()
/tmp/ipykernel_2718362/1791768742.py:1: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.pvNameMatch()
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:661: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_2['name_len'] = pv_2['name_mod'].str.len()


0
10
20
30
40
50
60
70
80
90
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000


/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:707: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pv_4['name_len'] = pv_4['name_mod'].str.len()


0
10
20
30
40
50
60
70
80
90
0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000
75000


# Read historical links

Note that the file Links_current.txt is renamed to Links_2020.txt

#### readHistLinks(ctry):
1. Loop through Links_*.txt and extract active links which have children firms in the target country, save them as Ownership/GB/GB_links_*.csv.
2. For Links_2020.txt, extract archived links which have children firms in the target country, save it as Ownership/GB/GB_links_archived.csv.

In [ ]:
processHist.readHistLinks(ctry)

#### readHistEntities(ctry):

3. Read Entities.txt and extract firms in the target country, save it as Ownership/GB/entities_GB.csv.

In [ ]:
processHist.readHistEntities(ctry)

#### readHistEPVY():
Read Entities.txt and extract EPVY entities, save it as Ownership/entities_epvy.csv.

In [ ]:
processHist.readHistEPVY()

In [ ]:
processHist.modHistEPVY()

/tmp/ipykernel_3476714/3068699887.py:1: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.modHistEPVY()


#### splitHistLinks(ctry):

4. Extract links with direct % > 50% and save those as Ownership/GB/GB_links_dir50_*.csv.
    - (* can be 1999-2020 or archived)
5. Extract links with DUO50C and save those as Ownership/GB/GB_links_duo50_*.csv .
6. Extract links with shareholders in the group (E, P, V, Y) and save those as Ownership/GB/GB_links_inv_*.csv.
    - E: Mutual & Pension Fund/Nominee/Trust/Trustee
    - P: Private equity firms
    - V: Venture capital
    - Y: Hedge fund
7. Extract subsidiaries with foreign GUO50 or GUO50C and save those as Ownership/GB/fg_uo_GB_*.csv.
8. Extract subsidiaries with foreign GUO25 or GUO25C and save those as Ownership/GB/fg_uo_25_GB_*.csv

In [ ]:
processHist.splitHistSpecialLinks(ctry)

In [3]:
processHist.splitHistLinks(ctry, dir50=True, duo50=True, inv=True)

/tmp/ipykernel_4121580/378976118.py:1: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitHistLinks(ctry, dir50=False, duo50=False, inv=True)


2007


/tmp/ipykernel_4121580/378976118.py:1: DtypeWarning: Columns (4,6,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitHistLinks(ctry, dir50=False, duo50=False, inv=True)


2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
archived


/tmp/ipykernel_4121580/378976118.py:1: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitHistLinks(ctry, dir50=False, duo50=False, inv=True)


# Process historical links
#### mergeHistLinks(ctry, link_type)
1. Loop through all the years and the archived file for the link type, record all the subsidiary and shareholder pairs.
2. Construct a dataframe using all the pairs as index, and set the first year and last year as the columns.
3. Loop through all the years again, and update the first year and last year for the pair.
    - If a link appears in year y, it means that it is active in that year so the last year is at least y.
4. Do the same for the archived file.
5. Save this file which contains the first year and the last year for all the pairs as Ownership/GB/GB_links_*_final.csv (* can be inv or dir50 or duo50).

In [4]:
for link_type in ['inv', 'duo50', 'dir50']:
    processHist.mergeHistLinks(ctry, link_type)

2007 99623
2008 133198
2009 159530
2010 190570
2011 236208
2012 280155
2013 319825
2014 354602
2015 390647
2016 423858
2017 458653
2018 491068
2019 516827
2020 532808
archived 629044
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020


#### extractHistLinks(ctry, link_type)
6. For EPVY backed links, groupby subsidiary and calculate the number of parents for each year. Save it as Ownership/GB/GB_links_inv_extract.csv.
7. For dir50 and duo50, groupby subsidiary and find their parents for each year. 
8. Forward fill the parents if missing.
9. Extract links for each year as Ownership/GB/GB_links_*link-type_extract_*.csv (*link-type can be dir50 or duo50, * is the year).

In [5]:
for link_type in ['inv', 'duo50', 'dir50']:
    processHist.extractHistLinks(ctry, link_type)

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)
/tmp/ipykernel_4121580/2992814864.py:5: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.extractHistLinks(ctry, link_type)


1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
1999 2
2000 12
2001 16
2002 57
2003 133
2004 817
2005 2541
2006 14689
2007 1496474
2008 1639841
2009 1707928
2010 1830752
2011 2054538
2012 2621164
2013 4709579
2014 4888358
2015 5089958
2016 5260277
2017 5248788
2018 5129099
2019 4858315
2020 4944478


/tmp/ipykernel_4121580/2992814864.py:5: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.extractHistLinks(ctry, link_type)


Number of duplicates: 5252
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
1999 18253
2000 77335
2001 152623
2002 367003
2003 482088
2004 727534
2005 953387
2006 1260328
2007 2491653
2008 2031382
2009 2150801
2010 2297386
2011 2564794
2012 3196033
2013 5750107
2014 5937592
2015 6162567
2016 6348482
2017 6352328
2018 6204147
2019 5868479
2020 5929526


#### splitBackedLinks(ctry)
From Ownership/GB/GB_links_inv_final.csv, do the same extract procedure for each of the E, P, V, Y and save them as Ownership/GB/GB_links_*_extract.csv.

In [3]:
# processHist.splitBackedLinks(ctry) # only run for PV

In [ ]:
# processHist.splitBackedLinks(ctry, fd=True)

In [2]:
# processHist.splitBackedLinks(ctry, fd=False, ori=True)

In [2]:
processHist.splitBackedLinks(ctry=ctry, fd=False, ori=False, nff=True)

/tmp/ipykernel_773948/3465473917.py:1: DtypeWarning: Columns (1,3,5,9,11,13,15,16,17,19,21,22,23,24,25,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=False, nff=True)
/tmp/ipykernel_773948/3465473917.py:1: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=False, nff=True)
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/indexing.py:719: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  indexer = self._get_setitem_indexer(key)
/tmp/ipykernel_773948/3465473917.py:1: DtypeWarning: Columns (4,6,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=Fal

2007 2006
2007 2007
2007 2004
2007 2005
2007 2003
2007 2002
2007 2001
2007 2000
2007 1999
2008 2008
2008 2007
2008 2006
2008 2005
2008 2004
2008 2003
2008 2000
2008 2001
2008 2002
2008 1999
2009 2008
2009 2009
2009 2007
2009 2006
2009 2005
2009 2004
2009 2003
2009 2001
2009 2000
2009 2002
2009 1999
2010 2010
2010 2007
2010 2009
2010 2008
2010 2004
2010 2006
2010 2005
2010 2003
2010 2001
2010 2000
2010 2002
2011 2011
2011 2010
2011 2009
2011 2008
2011 2007
2011 2004
2011 2006
2011 2005
2011 2003
2011 2001
2011 2002
2012 2012
2012 2010
2012 2009
2012 2011
2012 2007
2012 2008
2012 2006
2012 2003
2012 2004
2012 2005
2012 2002
2013 2013
2013 2010
2013 2012
2013 2011
2013 2007
2013 2009
2013 2008
2013 2006
2013 2004
2013 2003
2013 2005
2013 2002
2014 2014
2014 2013
2014 2011
2014 2012
2014 2007
2014 2010
2014 2008
2014 2004
2014 2005
2014 2009
2014 2006
2014 2002
2015 2015
2015 2012
2015 2013
2015 2014
2015 2011
2015 2010
2015 2008
2015 2005
2015 2009
2015 2006
2015 2007
2015 2002
2016 2016


/tmp/ipykernel_773948/3465473917.py:1: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=False, nff=True)


archived 2008
archived 2002
archived 2003
archived 2004
archived 2005
archived 2007
archived 2009
archived 2010
archived 2011
archived 2012
archived 2013
archived 2014
archived 2015
archived 2016
archived 2017
archived 2018
archived 2019
archived 2006
archived 2020
archived 2001
archived 2000
archived 1999


In [3]:
processHist.splitBackedLinks(ctry=ctry, fd=False, ori=True, nff=True)

/tmp/ipykernel_773948/2063244762.py:1: DtypeWarning: Columns (1,3,5,9,11,13,15,16,17,19,21,22,23,24,25,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=True, nff=True)
/tmp/ipykernel_773948/2063244762.py:1: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=True, nff=True)
/tmp/ipykernel_773948/2063244762.py:1: DtypeWarning: Columns (4,6,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=True, nff=True)


2007 2006
2007 2007
2007 2004
2007 2005
2007 2003
2007 2002
2007 2001
2007 2000
2007 1999
2008 2008
2008 2007
2008 2006
2008 2005
2008 2004
2008 2003
2008 2000
2008 2001
2008 2002
2008 1999
2009 2008
2009 2009
2009 2007
2009 2006
2009 2005
2009 2004
2009 2003
2009 2001
2009 2000
2009 2002
2009 1999
2010 2010
2010 2007
2010 2009
2010 2008
2010 2004
2010 2006
2010 2005
2010 2003
2010 2001
2010 2000
2010 2002
2011 2011
2011 2010
2011 2009
2011 2008
2011 2007
2011 2004
2011 2006
2011 2005
2011 2003
2011 2001
2011 2002
2012 2012
2012 2010
2012 2009
2012 2011
2012 2007
2012 2008
2012 2006
2012 2003
2012 2004
2012 2005
2012 2002
2013 2013
2013 2010
2013 2012
2013 2011
2013 2007
2013 2009
2013 2008
2013 2006
2013 2004
2013 2003
2013 2005
2013 2002
2014 2014
2014 2013
2014 2011
2014 2012
2014 2007
2014 2010
2014 2008
2014 2004
2014 2005
2014 2009
2014 2006
2014 2002
2015 2015
2015 2012
2015 2013
2015 2014
2015 2011
2015 2010
2015 2008
2015 2005
2015 2009
2015 2006
2015 2007
2015 2002
2016 2016


/tmp/ipykernel_773948/2063244762.py:1: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=False, ori=True, nff=True)


archived 2008
archived 2002
archived 2003
archived 2004
archived 2005
archived 2007
archived 2009
archived 2010
archived 2011
archived 2012
archived 2013
archived 2014
archived 2015
archived 2016
archived 2017
archived 2018
archived 2019
archived 2006
archived 2020
archived 2001
archived 2000
archived 1999


In [4]:
processHist.splitBackedLinks(ctry=ctry, fd=True, ori=False, nff=True)

/tmp/ipykernel_773948/686173957.py:1: DtypeWarning: Columns (1,3,5,9,11,13,15,16,17,19,21,22,23,24,25,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=True, ori=False, nff=True)
/tmp/ipykernel_773948/686173957.py:1: DtypeWarning: Columns (4,6,14,15,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=True, ori=False, nff=True)


2007 2006
2007 2007
2007 2004
2007 2005
2007 2003
2007 2002
2007 2001
2007 2000
2007 1999
2008 2008
2008 2007
2008 2006
2008 2005
2008 2004
2008 2003
2008 2000
2008 2001
2008 2002
2008 1999
2009 2008
2009 2009
2009 2007
2009 2006
2009 2005
2009 2004
2009 2003
2009 2001
2009 2000
2009 2002
2009 1999
2010 2010
2010 2007
2010 2009
2010 2008
2010 2004
2010 2006
2010 2005
2010 2003
2010 2001
2010 2000
2010 2002
2011 2011
2011 2010
2011 2009
2011 2008
2011 2007
2011 2004
2011 2006
2011 2005
2011 2003
2011 2001
2011 2002
2012 2012
2012 2010
2012 2009
2012 2011
2012 2007
2012 2008
2012 2006
2012 2003
2012 2004
2012 2005
2012 2002
2013 2013
2013 2010
2013 2012
2013 2011
2013 2007
2013 2009
2013 2008
2013 2006
2013 2004
2013 2003
2013 2005
2013 2002
2014 2014
2014 2013
2014 2011
2014 2012
2014 2007
2014 2010
2014 2008
2014 2004
2014 2005
2014 2009
2014 2006
2014 2002
2015 2015
2015 2012
2015 2013
2015 2014
2015 2011
2015 2010
2015 2008
2015 2005
2015 2009
2015 2006
2015 2007
2015 2002
2016 2016


/tmp/ipykernel_773948/686173957.py:1: DtypeWarning: Columns (16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.splitBackedLinks(ctry=ctry, fd=True, ori=False, nff=True)


archived 2008
archived 2002
archived 2003
archived 2004
archived 2005
archived 2007
archived 2009
archived 2010
archived 2011
archived 2012
archived 2013
archived 2014
archived 2015
archived 2016
archived 2017
archived 2018
archived 2019
archived 2006
archived 2020
archived 2001
archived 2000
archived 1999


#### propagateHistLinks(ctry)
1. For each year, exclude those links that contain inactive parents.
2. Include only links that the parents are industrial companies (from entities_GB.csv).
3. Extract parents that are not subsidiaries of others as ultimate parents. Set the ultimate parents for these firms as themselves.
4. For all the subsidiaries, initialize the ultimate parent as their direct parent.
4. Combine ultimate parents and all the subsidiaries in a single dataframe.
5. Iterate n times, each time pass the ultimate parent of the parent to its child.
6. Do the propagation for dir50 and duo50.
    - For duo50, we still need to do propagation since we are merging multiple years and there can be missing duo50 links if it was not updated properly.
7. Exclude subsidiaries in dir50 that are either the subsidiaries or the shareholders in duo50.
8. If an ultimate parent is a subsidiary in some links in duo50, we replace the ultimate parent to be the true ultimate parent in duo50.
9. We combine dir50 and duo50 and save it as Ownership/GB/comb_GB_*.csv (* is the year).


In [4]:
processHist.propagateHistLinks(ctry)

/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:1182: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  ind_set, inv_set, entity = getEntitiesSets(ctry)
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:1182: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  ind_set, inv_set, entity = getEntitiesSets(ctry)
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:318: DtypeWarning: Columns (6,8,14,22,25,27,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  df_header = readWRDSHeader(ctry)


2020
2019
2018
2017
2016
2015
2014
2013
2012
2011
2010
2009
2008
2007
2006
2005
2004
2003
2002
2001
2000
1999


In [2]:
# merge links, then merge Zephyr
df = pd.read_csv(fin_hist_fd +  ctry + '/' + ctry + '_filter1_merge_cons_yr_panel_ind.csv')

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6,8,105,107,109,113,115,117,119,129,135,136) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_header = processHist.getMergedHeader(ctry)

/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:318: DtypeWarning: Columns (6,8,14,22,25,27,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  df_header = readWRDSHeader(ctry)


In [4]:
df = processHist.mergeLinks(df, df_header, ctry=ctry, start_yr=2002)

/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:1301: DtypeWarning: Columns (3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  ind_set, inv_set, entity_copy = getEntitiesSets(ctry)
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:1301: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  ind_set, inv_set, entity_copy = getEntitiesSets(ctry)
/tmp/ipykernel_3454682/452302218.py:1: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  df = processHist.mergeLinks(df, df_header, ctry=ctry, start_yr=2002)


2002


/tmp/ipykernel_3454682/452302218.py:1: DtypeWarning: Columns (11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  df = processHist.mergeLinks(df, df_header, ctry=ctry, start_yr=2002)


2003


/tmp/ipykernel_3454682/452302218.py:1: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  df = processHist.mergeLinks(df, df_header, ctry=ctry, start_yr=2002)


...

/tmp/ipykernel_3454682/452302218.py:1: DtypeWarning: Columns (15,16,17,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  df = processHist.mergeLinks(df, df_header, ctry=ctry, start_yr=2002)


2013
2014
2015
2016
2017
2018
2019


/tmp/ipykernel_3454682/452302218.py:1: DtypeWarning: Columns (13,14,15,16,17,18,19,20,21,22,23) have mixed types.Specify dtype option on import or set low_memory=False.
  df = processHist.mergeLinks(df, df_header, ctry=ctry, start_yr=2002)


2020


/tmp/ipykernel_3454682/452302218.py:1: DtypeWarning: Columns (14,15,16,17,18,19,20,21,22) have mixed types.Specify dtype option on import or set low_memory=False.
  df = processHist.mergeLinks(df, df_header, ctry=ctry, start_yr=2002)


In [5]:
df.to_csv(fin_hist_fd +  ctry + '/' + ctry + '_filter1_merge_cons_yr_panel_ind_link.csv', index=False)

In [2]:
processHist.mergeZephyrHeader(ctry=ctry)

/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:318: DtypeWarning: Columns (6,8,14,22,25,27,36,37) have mixed types.Specify dtype option on import or set low_memory=False.
  df_header = readWRDSHeader(ctry)
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processZephyr.py:212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['final_stake'] = temp['final_stake_3']
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/nas/longleaf/home/yilu

In [2]:
df = pd.read_csv(fin_hist_fd +  ctry + '/' + ctry + '_filter1_merge_cons_yr_panel_ind_link.csv')

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6,8,105,107,109,113,115,117,119,129,135,136,139,140,141,142,143,144) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_header = pd.read_csv(des_hist_fd + ctry + '/' + ctry + '_header_zp_cpst.csv')

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,3,5,9,11,13,15,16,17,19,21,22,23,24,25,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
zp = processZephyr.Zephyr()
zp.extractDeals()
zp.removeDuplicates()
zp.readHandCheckData()

/nas/longleaf/home/yiluy/projects/BvDAnalysis/processZephyr.py:213: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['final_stake'] = temp['final_stake_3']
/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [5]:
df = processHist.mergeZephyr(df, df_header, zp)

In [6]:
df = processHist.concatRestructuring(df)

/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:1994: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_panel['closdate_year'] = df_panel['closdate_year'].astype(float)
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:1996: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_panel['first_yr'] = df_panel.groupby('bvdid')['closdate_year'].transform('min')
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:1997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [7]:
df = processHist.concatRTO(df, zp)

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:2139: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['tag_count'] = temp.groupby('tag_bvdid')['Deal Number'].transform('count')
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:2140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290


In [8]:
df = processHist.unifyID(df)

In [5]:
df = df[df.bvdid.str[:2]=='GB']

In [6]:
df.to_csv(fin_hist_fd +  ctry + '/' + ctry + '_filter1_merge_cons_yr_panel_ind_link_zp_rst.csv', index=False)

In [2]:
df = pd.read_csv(fin_hist_fd +  ctry + '/' + ctry + '_filter1_merge_cons_yr_panel_ind_link_zp_rst.csv')

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,8,36,105,107,109,113,115,117,119,129,135,139,140,141,142,143,144,154,155,156,157,158,160,161,162,163,164,166,167,168,169,170,173,174,175,176,177,180,181,182,183,185,186,187,188,189,191,192,193,194,195,197,198,199,200,201,203,204,205,206,207,209,210,211,212,213,215,216,217,218,219,222,223,224,225,226,227,228,229,230,231,233,235,237,243,246,247,248,249,253,255) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
processHist.updatePanelListedStatus(ctry=ctry)

/tmp/ipykernel_3506550/3939866359.py:1: DtypeWarning: Columns (1,3,5,9,11,13,15,16,17,19,21,22,23,24,25,26,30,32) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.updatePanelListedStatus(ctry=ctry)


finish reading header


/tmp/ipykernel_3506550/3939866359.py:1: DtypeWarning: Columns (8,105,107,109,113,115,117,119,129,135,139,140,141,142,143,144,154,155,156,157,158,160,161,162,163,164,166,167,168,169,170,173,174,175,176,177,180,181,182,183,185,186,187,188,189,191,192,193,194,195,197,198,199,200,201,203,204,205,206,207,209,210,211,212,213,215,216,217,218,219,222,223,224,225,226,227,228,229,230,231,233,235,237,243,246,247,248,249,253,255) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.updatePanelListedStatus(ctry=ctry)


finish reading panel data
finish merging with header
finish casting types
finish updating listed status
finish saving data


In [7]:
# processHist.labelPanel(ctry=ctry)

In [ ]:
processHist.filterPanel(ctry=ctry)

/tmp/ipykernel_3892583/2422198231.py:1: DtypeWarning: Columns (8,105,107,109,112,114,124,130,134,135,136,137,138,145,146,147,148,149,151,152,153,154,155,157,158,159,160,161,164,165,166,167,168,171,172,173,174,176,177,178,179,180,182,183,184,185,186,188,189,190,191,192,194,195,196,197,198,200,201,202,203,204,206,207,208,209,210,213,214,215,216,217,218,219,220,221,222,224,226,228,234,237,238,239,240,244,246,252,254,256,257,258) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.filterPanel(ctry=ctry)


In [2]:
df = pd.read_csv(fin_hist_fd +  ctry + '/' + ctry + '_filter1_merge_cons_yr_panel_ind_link_zp_rst_listed.csv')

# independent flag
temp_idx = (df['up_bvdid'].isna()) | (df['bvdid']==df['up_bvdid'])
df.loc[temp_idx, 'idpt_flag'] = 1

# adjust float variables based on the number of months in the financial report
# fl_cols = ['turn', 'opre', 'cost', 'gros', 'oope', 'fire', 'fiex', 'fipl', 'taxa', 'plat', 'exre', 'exex', 'extr', 'expt', 'mate', 'staf', 'depr', 'inte', 'rd', 'cf', 'av', 'ebta', 'pl', 'oppl', 'plbt']
fl_cols = ['opre']
temp_idx = df['nr_months']!=12
df.loc[temp_idx, fl_cols] = df.loc[temp_idx, fl_cols].mul(12 / df.loc[temp_idx, 'nr_months'], axis=0)

/nas/longleaf/rhel8/apps/anaconda/2021.11.ood/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8,105,107,109,112,114,124,130,134,135,136,137,138,145,146,147,148,149,151,152,153,154,155,157,158,159,160,161,164,165,166,167,168,171,172,173,174,176,177,178,179,180,182,183,184,185,186,188,189,190,191,192,194,195,196,197,198,200,201,202,203,204,206,207,208,209,210,213,214,215,216,217,218,219,220,221,222,224,226,228,234,237,238,239,240,244,246,252,254,256,257,258) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.loc[:, 'opre_flag'] = np.nan
opre_idx = ((df.turn/df.exchrate) >= 6500000) | ((df.opre/df.exchrate) >= 6500000)
toas_idx = (((df.cuas/df.exchrate) + (df.fias/df.exchrate)) >= 3260000)
opre_set = set(df[((df.opre.notna()) | (df.turn.notna())) & (opre_idx | toas_idx)].bvdid_r3.dropna().unique())
df.loc[df.bvdid_r3.isin(opre_set), 'opre_flag'] = 1

In [4]:
# get the set of firms that were idpt or up for at least one year
idpt_set = set(df[df['idpt_flag'] == 1].bvdid_r3.dropna().unique())

# get the set of firms that were industrial for at least one year
c_set = set(df[df['ent_final'] == 'C'].bvdid_r3.dropna().unique())

In [5]:
# filter the panel data
df = df[df.bvdid_r3.isin(idpt_set & opre_set & c_set)]

In [6]:
fl_cols = ['turn', 'cost', 'gros', 'oope', 'fire', 'fiex', 'fipl', 'taxa', 'plat', 'exre', 'exex', 'extr', 'expt', 'mate', 'staf', 'depr', 'inte', 'rd', 'cf', 'av', 'ebta', 'pl', 'oppl', 'plbt']
temp_idx = df['nr_months']!=12
df.loc[temp_idx, fl_cols] = df.loc[temp_idx, fl_cols].mul(12 / df.loc[temp_idx, 'nr_months'], axis=0)

In [7]:
# sort to make sure the data is sorted by bvdid_r3 and closdate_year
df.sort_values(['bvdid_r3', 'closdate_year'], inplace=True)

In [8]:
df.to_csv(fin_hist_fd +  ctry + '/' + ctry + '_filter1_merge_cons_yr_panel_ind_link_zp_rst_listed_label_filter.csv', index=False)

In [2]:
processHist.nextState(ctry=ctry)

/tmp/ipykernel_221398/3046384215.py:1: DtypeWarning: Columns (8,105,107,112,114,124,134,135,136,137,138,145,146,147,148,149,151,152,153,154,155,157,158,159,160,161,164,165,166,167,168,171,172,173,174,176,177,178,179,180,182,183,184,185,186,188,189,190,191,192,194,195,196,197,198,200,201,202,203,204,206,207,208,209,210,213,214,215,216,217,218,219,220,221,222,224,226,228,234,237,238,239,240,244,246,252,254,256,257,258) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.nextState(ctry=ctry)
/tmp/ipykernel_221398/3046384215.py:1: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  processHist.nextState(ctry=ctry)
/nas/longleaf/home/yiluy/projects/BvDAnalysis/processHist.py:2703: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

# Read historical descriptives - trade description, overviews, bankers, other advisors

In [ ]:
# processHist.readHistDesc(ctry)

In [ ]:
# df_trdes, df_ov, df_bk, df_adv = processHist.getHistDesc(ctry)